In [62]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [63]:
grade_states = ['A', 'B', 'C']
intel_states = ['High', 'Low']
study_states = ['Sufficient', 'Insufficient']
difficulty_states = ['Hard', 'Easy']
pass_states = ['Yes', 'No'] 

In [64]:
model = DiscreteBayesianNetwork([
  ('Intelligence', 'Grade'),
  ('Study Hours', 'Grade'),
  ('Difficulty', 'Grade'),
  ('Grade', 'Pass')
])

In [65]:
cpd_intel = TabularCPD(
    variable='Intelligence',
    variable_card=2,
    values=[
        [0.7], # high
        [0.3]  # low
    ],
    state_names={
        'Intelligence': intel_states
    }
)

cpd_study = TabularCPD(
    variable='Study Hours',
    variable_card=2,
    values=[
        [0.6], # Suff
        [0.4]  # Insuff
    ],
    state_names={
    'Study Hours': study_states
  }
)

cpd_diff = TabularCPD(
  variable='Difficulty',
  variable_card=2,
  values=[
    [0.4],
    [0.6]
  ],
  state_names={
    'Difficulty': difficulty_states
  }
)

# P(Grade | Intel, Study, Diff)
cpd_grade = TabularCPD(
  variable='Grade',
  variable_card=3,
  values=[
    [0.8, 0.6, 0.5, 0.3, 0.4, 0.2, 0.2, 0.1],  #  A             
    [0.2, 0.3, 0.4, 0.4, 0.4, 0.4, 0.3, 0.2],  #  B
    [0.0, 0.1, 0.1, 0.3, 0.2, 0.4, 0.5, 0.7]   #  C          
  ],
  evidence=['Intelligence', 'Study Hours', 'Difficulty'],
  evidence_card=[2, 2, 2],
  state_names={
    'Intelligence': intel_states,
    'Study Hours': study_states,
    'Difficulty': difficulty_states,
    'Grade': grade_states
  }
)

# P(Pass | Grade)
cpd_pass = TabularCPD(
  variable='Pass',
  variable_card=2,
  values=[
    [0.95, 0.8, 0.5], # yes
    [0.05, 0.2, 0.5]  # no
  ],
  evidence=['Grade'],
  evidence_card=[3],
  state_names={
    'Grade': grade_states,
    'Pass': pass_states
  }
)

In [66]:
model.add_cpds(cpd_diff, cpd_intel, cpd_grade, cpd_pass, cpd_study)

In [67]:
inference = VariableElimination(model)

In [68]:
# Probability that the student passes the exam, given: StudyHours = Sufficient, Difficulty = Hard
result1 = inference.query(variables=['Pass'], evidence={'Study Hours': 'Sufficient', 'Difficulty': 'Hard'})
print(result1)

+-----------+-------------+
| Pass      |   phi(Pass) |
+===========+=============+
| Pass(Yes) |      0.8840 |
+-----------+-------------+
| Pass(No)  |      0.1160 |
+-----------+-------------+


In [69]:
# What is the probability that the student has High Intelligence, given: Pass = Yes
result2 = inference.query(variables=['Intelligence'], evidence={'Pass': 'Yes'})
print(result2)

+--------------------+---------------------+
| Intelligence       |   phi(Intelligence) |
+====================+=====================+
| Intelligence(High) |              0.7380 |
+--------------------+---------------------+
| Intelligence(Low)  |              0.2620 |
+--------------------+---------------------+
